In [2]:
## Standard libraries
import os
import math
import numpy as np
import time
from fastcore.all import *
from nbdev.showdoc import *

# Configure environment
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false' # Tells Jax not to hog all of the memory to this process.

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.auto import tqdm, trange

import torch
import autometric
import dmae

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Curvature via Pullback Metrics
> How much does the metric give?

A good test of a metric is whether it can accurately reconstruct a manifold's curvature. Here we try this with the pullback metrics from 1) the Off-Manifolder, and 2) our distance matching encoder.

Neither of these is a classic 'intrinsic' metric for anything like intrinsic coords. Both will come from an embedding of the manifold in a dimension greater than its intrinsic dimension. The encoder pullback metric won't even have full rank. And the off-manifolder pullback starts from (e.g.) a hemisphere in 3d and wraps noise around the extra dimensions.

What both metrics might nonetheless preserve is some sense of *placement* on the manifold. The Off-Manifolder has, built in, a notion of tangent and normal directions. The encoder pullback knows which directions are noise.

**Hypothesis**: 
1. The Off-Manifolder adds an extra dimension to the manifold, and makes it highly negatively curved -- but perhaps only in the directions normal to the manifold. If we recover Ricci curvatures we might find that, for the hemisphere, the two directions tangent to the manifold have the correct Ricci curvatures, e.g. moving along them creates the curvature of the manifold, but I doubt it. The curvature is already there.
2. The encoder pullback metric shows only which dimensions of ambient space are compressed, not which are expanded. It's possible the pseudoinverse of the Jacobian would have the requisite information.

# Machinery

We start by defining the hemisphere, the Off-Manifolder pullback, and the encoder.

In [3]:
from autometric.datasets import Hemisphere
from autometric.off_manifold import OffManifolderLinear

INFO: Using pytorch backend


In [4]:
H = Hemisphere(3000)
H_highd = Hemisphere(3000, rotation_dimension = 10)

/home/piriac/Pumberton/Workshop/23-Geometric-AE/dmae/src/autometric/src/autometric/datasets.py:133: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data)


In [5]:
offy = OffManifolderLinear(H.X, folding_dim = 10, density_k = 5, density_tol = 0.1, density_exponential = 4)

In [6]:
from dmae.models.unified_model import GeometricAE
from dmae.models.distance_matching import DistanceMatching

ModuleNotFoundError: No module named 'dmae.models'

In [7]:
import dmae

In [11]:
help(OffManifolderLinear)

Help on class OffManifolderLinear in module autometric.off_manifold:

class OffManifolderLinear(builtins.object)
 |  OffManifolderLinear(X, density_loss_function=None, folding_dim=10, density_k=5, density_tol=0.1, density_exponential=4)
 |  
 |  Folds points off manifold into higher dimensions using random matrices.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, X, density_loss_function=None, folding_dim=10, density_k=5, density_tol=0.1, density_exponential=4)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  density_loss(self, points)
 |  
 |  immersion(self, points)
 |  
 |  pullback_metric(self, points)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables
 |  
 |  __weakref__
 |      list of weak references to the object



# Results

# Conclusion